In [ ]:
# Import the needed libraries
import pandas as pd

In [ ]:
# Load the data
unaligned_data = pd.read_csv("DataPreparation/CSVData/UnalignedData.csv")
aligned_data = pd.read_csv("DataPreparation/CSVData/AlignedData.csv")
label_df = pd.read_csv("DataPreparation/CSVData/LabelData.csv")

In [ ]:
# Concatenate the data
df = pd.concat([unaligned_data, aligned_data, label_df], axis=1)

In [ ]:
# Remove the rows with missing values
df = df.dropna(inplace=True)
df.head()